# hotel-id-2021-fgvc8

In [ ]:
import os
import sys
import fastai
import numpy as np 


from fastai.imports import *
from fastai.vision import *
from fastai.vision.all import *
from torchvision.models import vgg16_bn
from tqdm import tqdm
from sklearn import preprocessing

In [ ]:
## Defining Directories
work_dir = Path('/kaggle/working/')
path = Path('../input/hotel-id-2021-fgvc8')

sample_sub = path / 'sample_submission.csv'
labels = path / 'train.csv'

In [ ]:
## Reading csv
train_data = pd.read_csv(labels)
df_sample = pd.read_csv(sample_sub)


In [ ]:
train_data.head(5)

In [ ]:
train_data['image_path']= train_data['chain'].astype(str)  + '/' + train_data['image'].astype(str)
train_data[:3]

In [ ]:
len_data = len(train_data.value_counts())

print("data Length:", len_data)
print("Hotel Chains:",train_data['chain'].unique())
chain_len = len(train_data['chain'].unique())-1
chain_len

## Hotels per Chain

In [ ]:
hotel_per_chain={}
chains = train_data['chain'].unique()
for chain_id in chains:
    hotel_per_chain[chain_id] = train_data[train_data['chain']==chain_id]['hotel_id'].nunique()

bar = plt.bar(x=hotel_per_chain.keys(),height=hotel_per_chain.values())
plt.xlabel("Chain ID")
plt.ylabel("Count")
plt.title("Hotels per Chain")

## Images per Hotel

In [ ]:
image_per_hotel=[]
hotels_id = train_data['hotel_id'].unique()
for hotel_id in hotels_id:
    image_per_hotel.append(train_data[train_data['hotel_id']==hotel_id]['image'].nunique())

    
hotel_image_df = pd.DataFrame({"hotel_id":map(str,hotels_id),"image_count":image_per_hotel})
hotel_image_df.sort_values(by="image_count",ascending=False,inplace=True)  

plt.figure(figsize=[15,15])

plt.bar(x=hotel_image_df["hotel_id"],height=hotel_image_df["image_count"])
plt.xlabel("Hotel ID")
# 
plt.ylabel("Image Count")
plt.title("Hotels and their image count")

## Dataset samples


In [ ]:
train_dir = '../input/hotel-id-2021-fgvc8/train_images'
dataset = ImageDataLoaders.from_df(df = train_data[['image_path', 'chain']],
                                   path = train_dir,
                                   folder = '.',
                                   item_tfms=Resize(128, method='pad', pad_mode='reflection'),
                                   valid_pct=0.2,
                                   bs=32)

In [ ]:
dataset.show_batch()